In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'municpios-ibge:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4954091%2F8341005%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240510%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240510T171235Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9571c3a5f1f42e98f723f7b1a917832958a513c826fcb7ccba1b92deaff8d94ae26c1cdced7cd6807ca9ab64841f95b2ba95ca41edc788f96ceed9d9d0bc567c2bdc197c965061f5bcde56b260fef5841d3568fda2242172b30787b3db362e8264949a95e6ec6ae920eb0691903cf2ed4804ddb55dfcb957c0108eae1df208df92253cff3b586757363b0e4497d10f333a00ad19e8fe94669edcb440de492b5967737f9ea5b3ef1d2e2fe3c1c18e91e3f7de705a485091d5a3eee055e2d7e0f8836cd23124f8fccc1cb3ac27c515231ceacf210891eb02a97e5099446c57618f3d44d9e2912ced6b696ea621ca48b4438471f5bba941e71c6d3a63061c633e1d,bitos-bahia-e-paran-entre-2021-e-2023:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4981577%2F8377674%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240510%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240510T171235Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8f407811680feed865ea6e08148141975d2059cfe86526cb47f89e13c3d25496569698b29e432afa58cb738d01fb8c389061f4de272bd33256e95373bcfa5d90eb60378dc03d54a57009c78e86f81bd6fae744e7c2d9e470d2e21ffd7e823e2e056474d6c6dfd95703d5aa512150b2498c7ee5708dc17703ef088fe6811d4f9ad3acf2da8b61c2f4d8b35702d2f9ae6fc728e69e61d700ad3a696ae823b80ace1b48978cdb8357366cac95d318c58be7f561dade58b0c07e3f63b673c64a34520693932db7fab840db16cf6fa5330af456b7a8d1d28691f7f65a72cf2fc65194743e0db4077b7a41ec2934e3bae1274dda068e999ebddbbe827fa37d52788e68'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 77982 bytes downloaded
Downloaded and uncompressed: municpios-ibge
[==================================================] 24946214 bytes downloaded
Downloaded and uncompressed: bitos-bahia-e-paran-entre-2021-e-2023
Data source import complete.


In [2]:
# Importando bibliotecas
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Criar dicinários com definicões dos códigos CID das causas
codigo_suicidio = {
    "X60-X84": {  # Group X60 to X84
        "X060": "Auto-intoxicação p/exposição intencional a analgesicos antipiréticosanti-reumáticos não-opiáceos",
        "X061": "Auto-intox p/ exp intenc droga anticonvulsivante sedat hipnótico antiparkinsoniano psicotropico ncop",
        "X062": "Auto-intoxicação p/ exposição intencional narcótico psicodisleptico (alucinógenos) ncop",
        "X063": "Auto-intoxicação p/ exposição intencional outras subst farmacológicas de ação sobre o sna",
        "X064": "Auto-intoxicação p/exposição intencional outras drogas medicamentos e subst biológicas e as ne",
        "X065": "Auto-intoxicação voluntária por álcool",
        "X066": "Auto-intoxicação intencional por solventes orgânicos hidrocarbonetos halogenados e seus vapores",
        "X067": "Auto-intoxicação intencional por outros gases e vapores",
        "X068": "Auto-intoxicação por e exposição intencional a pesticidas",
        "X069": "Auto-intoxicação p/exposição intencional a outros produtos quimicos e substâncias nocivas ne",
        "X070": "Lesão autoprovocada intencionalmente por enforcamento estrangulamento e sufocação",
        "X071": "Lesão autoprovocada intencionalmente por afogamento e submersão",
        "X072": "Lesão autoprovocada intencionalmente por disparo de arma de fogo de mão",
        "X073": "Lesão autoprovocada intencionalmente p/ disparo espingarda carabina ou arma de fogo de maior calibre",
        "X074": "Lesão autoprovocada intencionalmente p/ disparo de arma de fogo e de arma de fogo não especificada",
        "X075": "Lesão autoprovocada intencionalmente por dispositivos explosivos",
        "X076": "Lesão autoprovocada intencionalmente pela fumaca pelo fogo e por chamas",
        "X077": "Lesão autoprovocada intencionalmente por vapor de água gases ou objetos quentes",
        "X078": "Lesão autoprovocada intencionalmente por objeto cortante ou penetrante",
        "X079": "Lesão autoprovocada intencionalmente por objeto contundente",
        "X080": "Lesão autoprovocada intencionalmente por precipitação de um lugar elevado",
        "X081": "Lesão autoprovocada intencionalmente por precipitação ou permanência diante de objeto em movimento",
        "X082": "Lesão autoprovocada intencionalmente por impacto de um veículo a motor",
        "X083": "Lesão autoprovocada intencionalmente por outros meios especificados",
        "X084": "Lesão autoprovocada intencionalmente por meios não especificados",
        "X60": "Auto-intoxicação p/exposição intencional a analgesicos antipiréticosanti-reumáticos não-opiáceos",
        "X61": "Auto-intox p/ exp intenc droga anticonvulsivante sedat hipnótico antiparkinsoniano psicotropico ncop",
        "X62": "Auto-intoxicação p/ exposição intencional narcótico psicodisleptico (alucinógenos) ncop",
        "X63": "Auto-intoxicação p/ exposição intencional outras subst farmacológicas de ação sobre o sna",
        "X64": "Auto-intoxicação p/exposição intencional outras drogas medicamentos e subst biológicas e as ne",
        "X65": "Auto-intoxicação voluntária por álcool",
        "X66": "Auto-intoxicação intencional por solventes orgânicos hidrocarbonetos halogenados e seus vapores",
        "X67": "Auto-intoxicação intencional por outros gases e vapores",
        "X68": "Auto-intoxicação por e exposição intencional a pesticidas",
        "X69": "Auto-intoxicação p/exposição intencional a outros produtos quimicos e substâncias nocivas ne",
        "X70": "Lesão autoprovocada intencionalmente por enforcamento estrangulamento e sufocação",
        "X71": "Lesão autoprovocada intencionalmente por afogamento e submersão",
        "X72": "Lesão autoprovocada intencionalmente por disparo de arma de fogo de mão",
        "X73": "Lesão autoprovocada intencionalmente p/ disparo espingarda carabina ou arma de fogo de maior calibre",
        "X74": "Lesão autoprovocada intencionalmente p/ disparo de arma de fogo e de arma de fogo não especificada",
        "X75": "Lesão autoprovocada intencionalmente por dispositivos explosivos",
        "X76": "Lesão autoprovocada intencionalmente pela fumaca pelo fogo e por chamas",
        "X77": "Lesão autoprovocada intencionalmente por vapor de água gases ou objetos quentes",
        "X78": "Lesão autoprovocada intencionalmente por objeto cortante ou penetrante",
        "X79": "Lesão autoprovocada intencionalmente por objeto contundente",
        "X80": "Lesão autoprovocada intencionalmente por precipitação de um lugar elevado",
        "X81": "Lesão autoprovocada intencionalmente por precipitação ou permanência diante de objeto em movimento",
        "X82": "Lesão autoprovocada intencionalmente por impacto de um veículo a motor",
        "X83": "Lesão autoprovocada intencionalmente por outros meios especificados",
        "X84": "Lesão autoprovocada intencionalmente por meios não especificados",
    },
    "X600-X840": {  # Group X600 to X840
        "X600": "Auto-intoxicação por e exposição, intencional, a analgésicos, antipiréticos e anti-reumáticos, não-opiáceos - residência",
        "X601": "Auto-intoxicação por e exposição, intencional, a analgésicos, antipiréticos e anti-reumáticos, não-opiáceos - habitação coletiva",
        "X602": "Auto-intoxicação por e exposição, intencional, a analgésicos, antipiréticos e anti-reumáticos, não-opiáceos - escolas, outras instituições e áreas de administração pública",
        "X603": "Auto-intoxicação por e exposição, intencional, a analgésicos, antipiréticos e anti-reumáticos, não-opiáceos - área para a prática de esportes e atletismo",
        "X604": "Auto-intoxicação por e exposição, intencional, a analgésicos, antipiréticos e anti-reumáticos, não-opiáceos - rua e estrada",
        "X605": "Auto-intoxicação por e exposição, intencional, a analgésicos, antipiréticos e anti-reumáticos, não-opiáceos - áreas de comércio e de serviços",
        "X606": "Auto-intoxicação por e exposição, intencional, a analgésicos, antipiréticos e anti-reumáticos, não-opiáceos - áreas industriais e em construção",
        "X607": "Auto-intoxicação por e exposição, intencional, a analgésicos, antipiréticos e anti-reumáticos, não-opiáceos - fazenda",
        "X608": "Auto-intoxicação por e exposição, intencional, a analgésicos, antipiréticos e anti-reumáticos, não-opiáceos - outros locais especificados",
        "X609": "Auto-intoxicação por e exposição, intencional, a analgésicos, antipiréticos e anti-reumáticos, não-opiáceos - local não especificado",
        "X610": "Auto-intoxicação por e exposição, intencional, a drogas anticonvulsivantes [antiepilépticos] sedativos, hipnóticos, antiparkinsonianos e psicotrópicos não classificados em outra parte - residência",
        "X611": "Auto-intoxicação por e exposição, intencional, a drogas anticonvulsivantes [antiepilépticos] sedativos, hipnóticos, antiparkinsonianos e psicotrópicos não classificados em outra parte - habitação coletiva",
        "X612": "Auto-intoxicação por e exposição, intencional, a drogas anticonvulsivantes [antiepilépticos] sedativos, hipnóticos, antiparkinsonianos e psicotrópicos não classificados em outra parte - escolas, outras instituições e áreas de administração pública",
        "X613": "Auto-intoxicação por e exposição, intencional, a drogas anticonvulsivantes [antiepilépticos] sedativos, hipnóticos, antiparkinsonianos e psicotrópicos não classificados em outra parte - área para a prática de esportes e atletismo",
        "X614": "Auto-intoxicação por e exposição, intencional, a drogas anticonvulsivantes [antiepilépticos] sedativos, hipnóticos, antiparkinsonianos e psicotrópicos não classificados em outra parte - rua e estrada",
        "X615": "Auto-intoxicação por e exposição, intencional, a drogas anticonvulsivantes [antiepilépticos] sedativos, hipnóticos, antiparkinsonianos e psicotrópicos não classificados em outra parte - áreas de comércio e de serviços",
        "X616": "Auto-intoxicação por e exposição, intencional, a drogas anticonvulsivantes [antiepilépticos] sedativos, hipnóticos, antiparkinsonianos e psicotrópicos não classificados em outra parte - áreas industriais e em construção",
        "X617": "Auto-intoxicação por e exposição, intencional, a drogas anticonvulsivantes [antiepilépticos] sedativos, hipnóticos, antiparkinsonianos e psicotrópicos não classificados em outra parte - fazenda",
        "X618": "Auto-intoxicação por e exposição, intencional, a drogas anticonvulsivantes [antiepilépticos] sedativos, hipnóticos, antiparkinsonianos e psicotrópicos não classificados em outra parte - outros locais especificados",
        "X619": "Auto-intoxicação por e exposição, intencional, a drogas anticonvulsivantes [antiepilépticos] sedativos, hipnóticos, antiparkinsonianos e psicotrópicos não classificados em outra parte - local não especificado",
        "X620": "Auto-intoxicação por e exposição, intencional, a narcóticos e psicodislépticos [alucinógenos] não classificados em outra parte - residência",
        "X621": "Auto-intoxicação por e exposição, intencional, a narcóticos e psicodislépticos [alucinógenos] não classificados em outra parte - habitação coletiva",
        "X622": "Auto-intoxicação por e exposição, intencional, a narcóticos e psicodislépticos [alucinógenos] não classificados em outra parte - escolas, outras instituições e áreas de administração pública",
        "X623": "Auto-intoxicação por e exposição, intencional, a narcóticos e psicodislépticos [alucinógenos] não classificados em outra parte - área para a prática de esportes e atletismo",
        "X624": "Auto-intoxicação por e exposição, intencional, a narcóticos e psicodislépticos [alucinógenos] não classificados em outra parte - rua e estrada",
        "X625": "Auto-intoxicação por e exposição, intencional, a narcóticos e psicodislépticos [alucinógenos] não classificados em outra parte - áreas de comércio e de serviços",
        "X626": "Auto-intoxicação por e exposição, intencional, a narcóticos e psicodislépticos [alucinógenos] não classificados em outra parte - áreas industriais e em construção",
        "X627": "Auto-intoxicação por e exposição, intencional, a narcóticos e psicodislépticos [alucinógenos] não classificados em outra parte - fazenda",
        "X628": "Auto-intoxicação por e exposição, intencional, a narcóticos e psicodislépticos [alucinógenos] não classificados em outra parte - outros locais especificados",
        "X629": "Auto-intoxicação por e exposição, intencional, a narcóticos e psicodislépticos [alucinógenos] não classificados em outra parte - local não especificado",
        "X630": "Auto-intoxicação por e exposição, intencional, a outras substâncias farmacológicas de ação sobre o sistema nervoso autônomo - residência",
        "X631": "Auto-intoxicação por e exposição, intencional, a outras substâncias farmacológicas de ação sobre o sistema nervoso autônomo - habitação coletiva",
        "X632": "Auto-intoxicação por e exposição, intencional, a outras substâncias farmacológicas de ação sobre o sistema nervoso autônomo - escolas, outras instituições e áreas de administração pública",
        "X633": "Auto-intoxicação por e exposição, intencional, a outras substâncias farmacológicas de ação sobre o sistema nervoso autônomo - área para a prática de esportes e atletismo",
        "X634": "Auto-intoxicação por e exposição, intencional, a outras substâncias farmacológicas de ação sobre o sistema nervoso autônomo - rua e estrada",
        "X635": "Auto-intoxicação por e exposição, intencional, a outras substâncias farmacológicas de ação sobre o sistema nervoso autônomo - áreas de comércio e de serviços",
        "X636": "Auto-intoxicação por e exposição, intencional, a outras substâncias farmacológicas de ação sobre o sistema nervoso autônomo - áreas industriais e em construção",
        "X637": "Auto-intoxicação por e exposição, intencional, a outras substâncias farmacológicas de ação sobre o sistema nervoso autônomo - fazenda",
        "X638": "Auto-intoxicação por e exposição, intencional, a outras substâncias farmacológicas de ação sobre o sistema nervoso autônomo - outros locais especificados",
        "X639": "Auto-intoxicação por e exposição, intencional, a outras substâncias farmacológicas de ação sobre o sistema nervoso autônomo - local não especificado",
        "X640": "Auto-intoxicação por e exposição, intencional, a outras drogas, medicamentos e substâncias biológicas e às não especificadas - residência",
        "X641": "Auto-intoxicação por e exposição, intencional, a outras drogas, medicamentos e substâncias biológicas e às não especificadas - habitação coletiva",
        "X642": "Auto-intoxicação por e exposição, intencional, a outras drogas, medicamentos e substâncias biológicas e às não especificadas - escolas, outras instituições e áreas de administração pública",
        "X643": "Auto-intoxicação por e exposição, intencional, a outras drogas, medicamentos e substâncias biológicas e às não especificadas - área para a prática de esportes e atletismo",
        "X644": "Auto-intoxicação por e exposição, intencional, a outras drogas, medicamentos e substâncias biológicas e às não especificadas - rua e estrada",
        "X645": "Auto-intoxicação por e exposição, intencional, a outras drogas, medicamentos e substâncias biológicas e às não especificadas - áreas de comércio e de serviços",
        "X646": "Auto-intoxicação por e exposição, intencional, a outras drogas, medicamentos e substâncias biológicas e às não especificadas - áreas industriais e em construção",
        "X647": "Auto-intoxicação por e exposição, intencional, a outras drogas, medicamentos e substâncias biológicas e às não especificadas - fazenda",
        "X648": "Auto-intoxicação por e exposição, intencional, a outras drogas, medicamentos e substâncias biológicas e às não especificadas - outros locais especificados",
        "X649": "Auto-intoxicação por e exposição, intencional, a outras drogas, medicamentos e substâncias biológicas e às não especificadas - local não especificado",
        "X650": "Auto-intoxicação voluntária por álcool  - residência",
        "X651": "Auto-intoxicação voluntária por álcool  - habitação coletiva",
        "X652": "Auto-intoxicação voluntária por álcool  - escolas, outras instituições e áreas de administração pública",
        "X653": "Auto-intoxicação voluntária por álcool  - área para a prática de esportes e atletismo",
        "X654": "Auto-intoxicação voluntária por álcool  - rua e estrada",
        "X655": "Auto-intoxicação voluntária por álcool  - áreas de comércio e de serviços",
        "X656": "Auto-intoxicação voluntária por álcool  - áreas industriais e em construção",
        "X657": "Auto-intoxicação voluntária por álcool  - fazenda",
        "X658": "Auto-intoxicação voluntária por álcool  - outros locais especificados",
        "X659": "Auto-intoxicação voluntária por álcool  - local não especificado",
        "X660": "Auto-intoxicação intencional por solventes orgânicos, hidrocarbonetos halogenados e seus vapores - residência",
        "X661": "Auto-intoxicação intencional por solventes orgânicos, hidrocarbonetos halogenados e seus vapores - habitação coletiva",
        "X662": "Auto-intoxicação intencional por solventes orgânicos, hidrocarbonetos halogenados e seus vapores - escolas, outras instituições e áreas de administração pública",
        "X663": "Auto-intoxicação intencional por solventes orgânicos, hidrocarbonetos halogenados e seus vapores - área para a prática de esportes e atletismo",
        "X664": "Auto-intoxicação intencional por solventes orgânicos, hidrocarbonetos halogenados e seus vapores - rua e estrada",
        "X665": "Auto-intoxicação intencional por solventes orgânicos, hidrocarbonetos halogenados e seus vapores - áreas de comércio e de serviços",
        "X666": "Auto-intoxicação intencional por solventes orgânicos, hidrocarbonetos halogenados e seus vapores - áreas industriais e em construção",
        "X667": "Auto-intoxicação intencional por solventes orgânicos, hidrocarbonetos halogenados e seus vapores - fazenda",
        "X668": "Auto-intoxicação intencional por solventes orgânicos, hidrocarbonetos halogenados e seus vapores - outros locais especificados",
        "X669": "Auto-intoxicação intencional por solventes orgânicos, hidrocarbonetos halogenados e seus vapores - local não especificado",
        "X670": "Auto-intoxicação intencional por outros gases e vapores - residência",
        "X671": "Auto-intoxicação intencional por outros gases e vapores - habitação coletiva",
        "X672": "Auto-intoxicação intencional por outros gases e vapores - escolas, outras instituições e áreas de administração pública",
        "X673": "Auto-intoxicação intencional por outros gases e vapores - área para a prática de esportes e atletismo",
        "X674": "Auto-intoxicação intencional por outros gases e vapores - rua e estrada",
        "X675": "Auto-intoxicação intencional por outros gases e vapores - áreas de comércio e de serviços",
        "X676": "Auto-intoxicação intencional por outros gases e vapores - áreas industriais e em construção",
        "X677": "Auto-intoxicação intencional por outros gases e vapores - fazenda",
        "X678": "Auto-intoxicação intencional por outros gases e vapores - outros locais especificados",
        "X679": "Auto-intoxicação intencional por outros gases e vapores - local não especificado",
        "X680": "Auto-intoxicação por e exposição, intencional, a pesticidas - residência",
        "X681": "Auto-intoxicação por e exposição, intencional, a pesticidas - habitação coletiva",
        "X682": "Auto-intoxicação por e exposição, intencional, a pesticidas - escolas, outras instituições e áreas de administração pública",
        "X683": "Auto-intoxicação por e exposição, intencional, a pesticidas - área para a prática de esportes e atletismo",
        "X684": "Auto-intoxicação por e exposição, intencional, a pesticidas - rua e estrada",
        "X685": "Auto-intoxicação por e exposição, intencional, a pesticidas - áreas de comércio e de serviços",
        "X686": "Auto-intoxicação por e exposição, intencional, a pesticidas - áreas industriais e em construção",
        "X687": "Auto-intoxicação por e exposição, intencional, a pesticidas - fazenda",
        "X688": "Auto-intoxicação por e exposição, intencional, a pesticidas - outros locais especificados",
        "X689": "Auto-intoxicação por e exposição, intencional, a pesticidas - local não especificado",
        "X690": "Auto-intoxicação por e exposição, intencional, a outros produtos químicos e substâncias nocivas não especificadas - residência",
        "X691": "Auto-intoxicação por e exposição, intencional, a outros produtos químicos e substâncias nocivas não especificadas - habitação coletiva",
        "X692": "Auto-intoxicação por e exposição, intencional, a outros produtos químicos e substâncias nocivas não especificadas - escolas, outras instituições e áreas de administração pública",
        "X693": "Auto-intoxicação por e exposição, intencional, a outros produtos químicos e substâncias nocivas não especificadas - área para a prática de esportes e atletismo",
        "X694": "Auto-intoxicação por e exposição, intencional, a outros produtos químicos e substâncias nocivas não especificadas - rua e estrada",
        "X695": "Auto-intoxicação por e exposição, intencional, a outros produtos químicos e substâncias nocivas não especificadas - áreas de comércio e de serviços",
        "X696": "Auto-intoxicação por e exposição, intencional, a outros produtos químicos e substâncias nocivas não especificadas - áreas industriais e em construção",
        "X697": "Auto-intoxicação por e exposição, intencional, a outros produtos químicos e substâncias nocivas não especificadas - fazenda",
        "X698": "Auto-intoxicação por e exposição, intencional, a outros produtos químicos e substâncias nocivas não especificadas - outros locais especificados",
        "X699": "Auto-intoxicação por e exposição, intencional, a outros produtos químicos e substâncias nocivas não especificadas - local não especificado",
        "X700": "Lesão autoprovocada intencionalmente por enforcamento, estrangulamento e sufocação - residência",
        "X701": "Lesão autoprovocada intencionalmente por enforcamento, estrangulamento e sufocação - habitação coletiva",
        "X702": "Lesão autoprovocada intencionalmente por enforcamento, estrangulamento e sufocação - escolas, outras instituições e áreas de administração pública",
        "X703": "Lesão autoprovocada intencionalmente por enforcamento, estrangulamento e sufocação - área para a prática de esportes e atletismo",
        "X704": "Lesão autoprovocada intencionalmente por enforcamento, estrangulamento e sufocação - rua e estrada",
        "X705": "Lesão autoprovocada intencionalmente por enforcamento, estrangulamento e sufocação - áreas de comércio e de serviços",
        "X706": "Lesão autoprovocada intencionalmente por enforcamento, estrangulamento e sufocação - áreas industriais e em construção",
        "X707": "Lesão autoprovocada intencionalmente por enforcamento, estrangulamento e sufocação - fazenda",
        "X708": "Lesão autoprovocada intencionalmente por enforcamento, estrangulamento e sufocação - outros locais especificados",
        "X709": "Lesão autoprovocada intencionalmente por enforcamento, estrangulamento e sufocação - local não especificado",
        "X710": "Lesão autoprovocada intencionalmente por afogamento e submersão - residência",
        "X711": "Lesão autoprovocada intencionalmente por afogamento e submersão - habitação coletiva",
        "X712": "Lesão autoprovocada intencionalmente por afogamento e submersão - escolas, outras instituições e áreas de administração pública",
        "X713": "Lesão autoprovocada intencionalmente por afogamento e submersão - área para a prática de esportes e atletismo",
        "X714": "Lesão autoprovocada intencionalmente por afogamento e submersão - rua e estrada",
        "X715": "Lesão autoprovocada intencionalmente por afogamento e submersão - áreas de comércio e de serviços",
        "X716": "Lesão autoprovocada intencionalmente por afogamento e submersão - áreas industriais e em construção",
        "X717": "Lesão autoprovocada intencionalmente por afogamento e submersão - fazenda",
        "X718": "Lesão autoprovocada intencionalmente por afogamento e submersão - outros locais especificados",
        "X719": "Lesão autoprovocada intencionalmente por afogamento e submersão - local não especificado",
        "X720": "Lesão autoprovocada intencionalmente por disparo de arma de fogo de mão - residência",
        "X721": "Lesão autoprovocada intencionalmente por disparo de arma de fogo de mão - habitação coletiva",
        "X722": "Lesão autoprovocada intencionalmente por disparo de arma de fogo de mão - escolas, outras instituições e áreas de administração pública",
        "X723": "Lesão autoprovocada intencionalmente por disparo de arma de fogo de mão - área para a prática de esportes e atletismo",
        "X724": "Lesão autoprovocada intencionalmente por disparo de arma de fogo de mão - rua e estrada",
        "X725": "Lesão autoprovocada intencionalmente por disparo de arma de fogo de mão - áreas de comércio e de serviços",
        "X726": "Lesão autoprovocada intencionalmente por disparo de arma de fogo de mão - áreas industriais e em construção",
        "X727": "Lesão autoprovocada intencionalmente por disparo de arma de fogo de mão - fazenda",
        "X728": "Lesão autoprovocada intencionalmente por disparo de arma de fogo de mão - outros locais especificados",
        "X729": "Lesão autoprovocada intencionalmente por disparo de arma de fogo de mão - local não especificado",
        "X730": "Lesão autoprovocada intencionalmente por disparo de espingarda, carabina, ou arma de fogo de maior calibre - residência",
        "X731": "Lesão autoprovocada intencionalmente por disparo de espingarda, carabina, ou arma de fogo de maior calibre - habitação coletiva",
        "X732": "Lesão autoprovocada intencionalmente por disparo de espingarda, carabina, ou arma de fogo de maior calibre - escolas, outras instituições e áreas de administração pública",
        "X733": "Lesão autoprovocada intencionalmente por disparo de espingarda, carabina, ou arma de fogo de maior calibre - área para a prática de esportes e atletismo",
        "X734": "Lesão autoprovocada intencionalmente por disparo de espingarda, carabina, ou arma de fogo de maior calibre - rua e estrada",
        "X735": "Lesão autoprovocada intencionalmente por disparo de espingarda, carabina, ou arma de fogo de maior calibre - áreas de comércio e de serviços",
        "X736": "Lesão autoprovocada intencionalmente por disparo de espingarda, carabina, ou arma de fogo de maior calibre - áreas industriais e em construção",
        "X737": "Lesão autoprovocada intencionalmente por disparo de espingarda, carabina, ou arma de fogo de maior calibre - fazenda",
        "X738": "Lesão autoprovocada intencionalmente por disparo de espingarda, carabina, ou arma de fogo de maior calibre - outros locais especificados",
        "X739": "Lesão autoprovocada intencionalmente por disparo de espingarda, carabina, ou arma de fogo de maior calibre - local não especificado",
        "X740": "Lesão autoprovocada intencionalmente por disparo de outra arma de fogo e de arma de fogo não especificada - residência",
        "X741": "Lesão autoprovocada intencionalmente por disparo de outra arma de fogo e de arma de fogo não especificada - habitação coletiva",
        "X742": "Lesão autoprovocada intencionalmente por disparo de outra arma de fogo e de arma de fogo não especificada - escolas, outras instituições e áreas de administração pública",
        "X743": "Lesão autoprovocada intencionalmente por disparo de outra arma de fogo e de arma de fogo não especificada - área para a prática de esportes e atletismo",
        "X744": "Lesão autoprovocada intencionalmente por disparo de outra arma de fogo e de arma de fogo não especificada - rua e estrada",
        "X745": "Lesão autoprovocada intencionalmente por disparo de outra arma de fogo e de arma de fogo não especificada - áreas de comércio e de serviços",
        "X746": "Lesão autoprovocada intencionalmente por disparo de outra arma de fogo e de arma de fogo não especificada - áreas industriais e em construção",
        "X747": "Lesão autoprovocada intencionalmente por disparo de outra arma de fogo e de arma de fogo não especificada - fazenda",
        "X748": "Lesão autoprovocada intencionalmente por disparo de outra arma de fogo e de arma de fogo não especificada - outros locais especificados",
        "X749": "Lesão autoprovocada intencionalmente por disparo de outra arma de fogo e de arma de fogo não especificada - local não especificado",
        "X750": "Lesão autoprovocada intencionalmente por dispositivos explosivos - residência",
        "X751": "Lesão autoprovocada intencionalmente por dispositivos explosivos - habitação coletiva",
        "X752": "Lesão autoprovocada intencionalmente por dispositivos explosivos - escolas, outras instituições e áreas de administração pública",
        "X753": "Lesão autoprovocada intencionalmente por dispositivos explosivos - área para a prática de esportes e atletismo",
        "X754": "Lesão autoprovocada intencionalmente por dispositivos explosivos - rua e estrada",
        "X755": "Lesão autoprovocada intencionalmente por dispositivos explosivos - áreas de comércio e de serviços",
        "X756": "Lesão autoprovocada intencionalmente por dispositivos explosivos - áreas industriais e em construção",
        "X757": "Lesão autoprovocada intencionalmente por dispositivos explosivos - fazenda",
        "X758": "Lesão autoprovocada intencionalmente por dispositivos explosivos - outros locais especificados",
        "X759": "Lesão autoprovocada intencionalmente por dispositivos explosivos - local não especificado",
        "X760": "Lesão autoprovocada intencionalmente pela fumaça, pelo fogo e por chamas - residência",
        "X760": "Lesão autoprovocada intencionalmente pela fumaça, pelo fogo e por chamas - residência",
        "X761": "Lesão autoprovocada intencionalmente pela fumaça, pelo fogo e por chamas - habitação coletiva",
        "X762": "Lesão autoprovocada intencionalmente pela fumaça, pelo fogo e por chamas - escolas, outras instituições e áreas de administração pública",
        "X763": "Lesão autoprovocada intencionalmente pela fumaça, pelo fogo e por chamas - área para a prática de esportes e atletismo",
        "X764": "Lesão autoprovocada intencionalmente pela fumaça, pelo fogo e por chamas - rua e estrada",
        "X765": "Lesão autoprovocada intencionalmente pela fumaça, pelo fogo e por chamas - áreas de comércio e de serviços",
        "X766": "Lesão autoprovocada intencionalmente pela fumaça, pelo fogo e por chamas - áreas industriais e em construção",
        "X767": "Lesão autoprovocada intencionalmente pela fumaça, pelo fogo e por chamas - fazenda",
        "X768": "Lesão autoprovocada intencionalmente pela fumaça, pelo fogo e por chamas - outros locais especificados",
        "X769": "Lesão autoprovocada intencionalmente pela fumaça, pelo fogo e por chamas - local não especificado",
        "X770": "Lesão autoprovocada intencionalmente por vapor de água, gases ou objetos quentes - residência",
        "X771": "Lesão autoprovocada intencionalmente por vapor de água, gases ou objetos quentes - habitação coletiva",
        "X772": "Lesão autoprovocada intencionalmente por vapor de água, gases ou objetos quentes - escolas, outras instituições e áreas de administração pública",
        "X773": "Lesão autoprovocada intencionalmente por vapor de água, gases ou objetos quentes - área para a prática de esportes e atletismo",
        "X774": "Lesão autoprovocada intencionalmente por vapor de água, gases ou objetos quentes - rua e estrada",
        "X775": "Lesão autoprovocada intencionalmente por vapor de água, gases ou objetos quentes - áreas de comércio e de serviços",
        "X776": "Lesão autoprovocada intencionalmente por vapor de água, gases ou objetos quentes - áreas industriais e em construção",
        "X777": "Lesão autoprovocada intencionalmente por vapor de água, gases ou objetos quentes - fazenda",
        "X778": "Lesão autoprovocada intencionalmente por vapor de água, gases ou objetos quentes - outros locais especificados",
        "X779": "Lesão autoprovocada intencionalmente por vapor de água, gases ou objetos quentes - local não especificado",
        "X780": "Lesão autoprovocada intencionalmente por objeto cortante ou penetrante - residência",
        "X781": "Lesão autoprovocada intencionalmente por objeto cortante ou penetrante - habitação coletiva",
        "X782": "Lesão autoprovocada intencionalmente por objeto cortante ou penetrante - escolas, outras instituições e áreas de administração pública",
        "X783": "Lesão autoprovocada intencionalmente por objeto cortante ou penetrante - área para a prática de esportes e atletismo",
        "X784": "Lesão autoprovocada intencionalmente por objeto cortante ou penetrante - rua e estrada",
        "X785": "Lesão autoprovocada intencionalmente por objeto cortante ou penetrante - áreas de comércio e de serviços",
        "X786": "Lesão autoprovocada intencionalmente por objeto cortante ou penetrante - áreas industriais e em construção",
        "X787": "Lesão autoprovocada intencionalmente por objeto cortante ou penetrante - fazenda",
        "X788": "Lesão autoprovocada intencionalmente por objeto cortante ou penetrante - outros locais especificados",
        "X789": "Lesão autoprovocada intencionalmente por objeto cortante ou penetrante - local não especificado",
        "X790": "Lesão autoprovocada intencionalmente por objeto contundente  - residência",
        "X791": "Lesão autoprovocada intencionalmente por objeto contundente  - habitação coletiva",
        "X792": "Lesão autoprovocada intencionalmente por objeto contundente  - escolas, outras instituições e áreas de administração pública",
        "X793": "Lesão autoprovocada intencionalmente por objeto contundente  - área para a prática de esportes e atletismo",
        "X794": "Lesão autoprovocada intencionalmente por objeto contundente  - rua e estrada",
        "X795": "Lesão autoprovocada intencionalmente por objeto contundente  - áreas de comércio e de serviços",
        "X796": "Lesão autoprovocada intencionalmente por objeto contundente  - áreas industriais e em construção",
        "X797": "Lesão autoprovocada intencionalmente por objeto contundente  - fazenda",
        "X798": "Lesão autoprovocada intencionalmente por objeto contundente  - outros locais especificados",
        "X799": "Lesão autoprovocada intencionalmente por objeto contundente  - local não especificado",
        "X800": "Lesão autoprovocada intencionalmente por precipitação de um lugar elevado - residência",
        "X801": "Lesão autoprovocada intencionalmente por precipitação de um lugar elevado - habitação coletiva",
        "X802": "Lesão autoprovocada intencionalmente por precipitação de um lugar elevado - escolas, outras instituições e áreas de administração pública",
        "X803": "Lesão autoprovocada intencionalmente por precipitação de um lugar elevado - área para a prática de esportes e atletismo",
        "X804": "Lesão autoprovocada intencionalmente por precipitação de um lugar elevado - rua e estrada",
        "X805": "Lesão autoprovocada intencionalmente por precipitação de um lugar elevado - áreas de comércio e de serviços",
        "X806": "Lesão autoprovocada intencionalmente por precipitação de um lugar elevado - áreas industriais e em construção",
        "X807": "Lesão autoprovocada intencionalmente por precipitação de um lugar elevado - fazenda",
        "X808": "Lesão autoprovocada intencionalmente por precipitação de um lugar elevado - outros locais especificados",
        "X809": "Lesão autoprovocada intencionalmente por precipitação de um lugar elevado - local não especificado",
        "X810": "Lesão autoprovocada intencionalmente por precipitação ou permanência diante de um objeto em movimento  - residência",
        "X811": "Lesão autoprovocada intencionalmente por precipitação ou permanência diante de um objeto em movimento  - habitação coletiva",
        "X812": "Lesão autoprovocada intencionalmente por precipitação ou permanência diante de um objeto em movimento  - escolas, outras instituições e áreas de administração pública",
        "X813": "Lesão autoprovocada intencionalmente por precipitação ou permanência diante de um objeto em movimento  - área para a prática de esportes e atletismo",
        "X814": "Lesão autoprovocada intencionalmente por precipitação ou permanência diante de um objeto em movimento  - rua e estrada",
        "X815": "Lesão autoprovocada intencionalmente por precipitação ou permanência diante de um objeto em movimento  - áreas de comércio e de serviços",
        "X816": "Lesão autoprovocada intencionalmente por precipitação ou permanência diante de um objeto em movimento  - áreas industriais e em construção",
        "X817": "Lesão autoprovocada intencionalmente por precipitação ou permanência diante de um objeto em movimento  - fazenda",
        "X818": "Lesão autoprovocada intencionalmente por precipitação ou permanência diante de um objeto em movimento  - outros locais especificados",
        "X819": "Lesão autoprovocada intencionalmente por precipitação ou permanência diante de um objeto em movimento  - local não especificado",
        "X820": "Lesão autoprovocada intencionalmente por impacto de um veículo a motor - residência",
        "X821": "Lesão autoprovocada intencionalmente por impacto de um veículo a motor - habitação coletiva",
        "X822": "Lesão autoprovocada intencionalmente por impacto de um veículo a motor - escolas, outras instituições e áreas de administração pública",
        "X823": "Lesão autoprovocada intencionalmente por impacto de um veículo a motor - área para a prática de esportes e atletismo",
        "X824": "Lesão autoprovocada intencionalmente por impacto de um veículo a motor - rua e estrada",
        "X825": "Lesão autoprovocada intencionalmente por impacto de um veículo a motor - áreas de comércio e de serviços",
        "X826": "Lesão autoprovocada intencionalmente por impacto de um veículo a motor - áreas industriais e em construção",
        "X827": "Lesão autoprovocada intencionalmente por impacto de um veículo a motor - fazenda",
        "X828": "Lesão autoprovocada intencionalmente por impacto de um veículo a motor - outros locais especificados",
        "X829": "Lesão autoprovocada intencionalmente por impacto de um veículo a motor - local não especificado",
        "X830": "Lesão autoprovocada intencionalmente por outros meios especificados - residência",
        "X831": "Lesão autoprovocada intencionalmente por outros meios especificados - habitação coletiva",
        "X832": "Lesão autoprovocada intencionalmente por outros meios especificados - escolas, outras instituições e áreas de administração pública",
        "X833": "Lesão autoprovocada intencionalmente por outros meios especificados - área para a prática de esportes e atletismo",
        "X834": "Lesão autoprovocada intencionalmente por outros meios especificados - rua e estrada",
        "X835": "Lesão autoprovocada intencionalmente por outros meios especificados - áreas de comércio e de serviços",
        "X836": "Lesão autoprovocada intencionalmente por outros meios especificados - áreas industriais e em construção",
        "X837": "Lesão autoprovocada intencionalmente por outros meios especificados - fazenda",
        "X838": "Lesão autoprovocada intencionalmente por outros meios especificados - outros locais especificados",
        "X839": "Lesão autoprovocada intencionalmente por outros meios especificados - local não especificado",
        "X840": "Lesão autoprovocada intencionalmente por meios não especificados - residência",
        "X841": "Lesão autoprovocada intencionalmente por meios não especificados - habitação coletiva",
        "X842": "Lesão autoprovocada intencionalmente por meios não especificados - escolas, outras instituições e áreas de administração pública",
        "X843": "Lesão autoprovocada intencionalmente por meios não especificados - área para a prática de esportes e atletismo",
        "X844": "Lesão autoprovocada intencionalmente por meios não especificados - rua e estrada",
        "X845": "Lesão autoprovocada intencionalmente por meios não especificados - áreas de comércio e de serviços",
        "X846": "Lesão autoprovocada intencionalmente por meios não especificados - áreas industriais e em construção",
        "X847": "Lesão autoprovocada intencionalmente por meios não especificados - fazenda",
        "X848": "Lesão autoprovocada intencionalmente por meios não especificados - outros locais especificados",
        "X849": "Lesão autoprovocada intencionalmente por meios não especificados - local não especificado",
    }
}

In [4]:
# Criar dicionário com colunas dos dataframes e definições
# As linhas COMENTADAS possuem informações que são contempladas por outras variáveis (colunas) ou não serão relevantes para o projeto
# Os comentários ao lado direito das linhas contemplam detalhes sobre as variáveis (colunas)

# FONTE: https://diaad.s3.sa-east-1.amazonaws.com/sim/Mortalidade_Geral+-+Estrutura.pdf

col_dict = {
    'SEXO': 'Sexo do falecido', # M - Masculino; F - Feminino; I - Ignorado
    'TPPOS': 'Óbito Investigado', # 1 - SIM: 2 - NÃO
    'CODMUNNATU': 'Código do município de naturalidade do falecido.', # Codificação do IBGE (?)
    'CODMUNOCOR': 'Código relativo ao município onde ocorreu o óbito', # Codificação do IBGE (?)
    'CODMUNRES':'Código do município de residência', # Codificação do IBGE (?) (Em caso de óbito fetal, considerar o município de residência da mãe)
    'NATURAL': 'País e Unidade da Federação onde falecido nasceu', # Estrangeiro tem número do país
    'NECROPSIA': 'Execução ou não de necropsia para confirmação do diagnóstico', # 1 - SIM; 2 - NÃO; 9 - IGNORADO
    'TPPOS': 'Óbito Investigado', # 1 - SIM; 2 - NÃO
    'OCUP': 'Tipo de trabalho que falecido desenvolveu na maior parte de sua vida produtivia', # De acordo com Classificação Brasileira de Ocupações - CBO 2002
    'QTDFILMORT': 'Número de filhos mortos', # Números de filhos mortos, sendo que 9 - IGNORADO (identificar se não há alguém com 9 filhos falecidos)
    'OBITOGRAV': 'Óbito na gravidez', # 1 - SIM; 2 - NÃO; 9 - Ignorado
    'RACACOR': 'Etnia do falecido', # 1 - BRANCA; 2 - PRETA; 3 - AMARELA; 4 - PARDA; 5 - INDÍGENA
    'GRAVIDEZ': 'Tipo de gravidez', # 1 - Única; 2 - Dupla; 3 - Tripla e mais; 9 - Ignorado
    'IDADE': 'Idade do falecido', # Idade do falecido em minutos, horas, dias, meses ou anos. (Idade: composto de dois subcampos. - O primeiro, de 1 dígito, indica a unidade da idade (se 1 = minuto, se 2 = hora, se 3 = mês, se 4 = ano, se = 5 idade maior que 100 anos). - O segundo, de dois dígitos, indica a quantidade de unidades: Idade menor de 1 hora: subcampo varia de 01 e 59 (minutos); De 1 a 23 Horas: subcampo varia de 01 a 23 (horas); De 24 horas e 29 dias: subcampo varia de 01 a 29 (dias); De 1 a menos de 12 meses completos: subcampo varia de 01 a 11 (meses); Anos - subcampo varia de 00 a 99; - 9 - ignorado)
    'DTOBITO': 'Data em que ocorreu óbito', # Formato ddmmaaaa (mesma informação do DATAOBITO)
    'DIFDATA': 'Diferença entre a data de óbito e data do recebimento original da DO', # ([DTOBITO] – [DTRECORIG]). (Números)
    'ESC': 'Escolaridade em ano', # 1 – Nenhuma; 2 – de 1 a 3 anos; 3 – de 4 a 7 anos; 4 – de 8 a 11 anos; 5 – 12 anos e mais; 9 – Ignorado
    'ESCFALAGR1': 'Escolaridade do falecido agregada', # 00 – Sem Escolaridade; 01 – Fundamental I Incompleto; 02 – Fundamental I Completo; 03 – Fundamental II Incompleto; 04 – Fundamental II Completo; 05 – Ensino Médio Incompleto; 06 – Ensino Médio Completo; 07 – Superior Incompleto; 08 – Superior Completo; 09 – Ignorado; 10 – Fundamental I Incompleto ou Inespecífico; 11 – Fundamental II Incompleto ou Inespecífico; 12 – Ensino Médio Incompleto ou Inespecífico
    'ESTCIV': 'Situação conjugal do falecido informada pelos familiares', # 1 – Solteiro; 2 – Casado; 3 – Viúvo; 4 – Separado judicialmente/divorciado; 5 – União estável; 9 – Ignorado)
    'ALTCAUSA': 'Indica se houve correção ou alteração da causa do óbito após investigação', #  1 - SIM; 2 - NÃO
    'ATESTADO': 'CIDs informados no atestado', # CID 10
    'CAUSABAS': 'CID de Causa Básica da DO', # CID 10
    'CIRCOBITO': 'Tipo de morte violenta ou circunstâncias em que se deu a morte não natural', # 1 – Acidente; 2 – Suicídio; 3 – Homicídio; 4 – Outros; 9 – Ignorado
    'LINHAA': 'Causa Terminal', # CIDs informados na Linha A da DO referente ao diagnóstico na Linha A da DO (causa terminal - doença ou estado mórbido que causou diretamente a morte). (Códigos CID 10)
    'LINHAB': 'Causa Antecedente ou Consequente', # CIDs informados na Linha B da DO referente ao diagnóstico na Linha B da DO (causa antecedente ou conseqüencial - estado mórbido, se existir, que produziu a causa direta da morte registrada na linha A). (Códigos CID 10)
    'LINHAC': 'Causa Antecedente ou Consequente', # CIDs informados na Linha C da DO referente ao diagnóstico na Linha C da DO (causa antecedente ou conseqüencial - estado mórbido, se existir, que produziu a causa direta da morte registrada na linha A). (Códigos CID 10)
    'LINHAD': 'Causa Básica', # CIDs informados na Linha D da DO referente ao diagnóstico na Linha D da DO (causa básica – estado mórbido, se existir, que produziu a causa direta da morte registrada na linha A). (Códigos CID 10)
    'LINHAII': 'Causa Contribuinte', # CIDs informados na Parte II da DO referente ao diagnóstico na Parte II da DO (causa contribuinte - outras condições significativas que contribuíram para a morte e que não entraram na cadeia definida na Parte I. (Códigos CID 10)

}


In [5]:
# Criar dicionário com csv dos municipios do ibge
import csv

# Carregar csv municipios IBGE
mun_ibge = pd.read_csv('/kaggle/input/municpios-ibge/mun_ibge.csv')

# Renomeie a coluna 'Cód Ibge' para 'Cód IBGE'
mun_ibge.rename(columns={'Cód IBGE': 'cod_mun'}, inplace=True)

# Visualizar head municipios ibge
mun_ibge.head()

,Unnamed: 0,UF,Município,cod_mun,Cód Região de Saúde,Nome da Região de Saúde
0,1,AC,Acrelândia,120001,12002,Baixo Acre e Purus
1,2,AC,Assis Brasil,120005,12001,Alto Acre
2,3,AC,Brasiléia,120010,12001,Alto Acre
3,4,AC,Bujari,120013,12002,Baixo Acre e Purus
4,5,AC,Capixaba,120017,12002,Baixo Acre e Purus


In [6]:
# Definir os índices apropriados como MultiIndex
mun_ibge.set_index(['UF', 'Município', 'cod_mun'], inplace=True)

# Criar DataFrame filtrado usando .loc[]
mun_bapr = mun_ibge.loc[mun_ibge.index.get_level_values('UF').isin(['BA', 'PR']), :]

# Resetar o índice para obter o DataFrame no formato desejado
mun_bapr.reset_index(inplace=True)

# Selecionar apenas as colunas desejadas
mun_bapr = mun_bapr[['UF', 'Município', 'cod_mun']]

# Verificar o novo DataFrame
print(mun_bapr)

     UF       Município  cod_mun
0    BA          Abaíra   290010
1    BA           Abaré   290020
2    BA       Acajutiba   290030
3    BA        Adustina   290035
4    BA       Água Fria   290040
..   ..             ...      ...
811  PR            Verê   412860
812  PR         Virmond   412865
813  PR        Vitorino   412870
814  PR  Wenceslau Braz   412850
815  PR          Xambrê   412880

[816 rows x 3 columns]


In [7]:
mun_bapr.head(816)

,UF,Município,cod_mun
0,BA,Abaíra,290010
1,BA,Abaré,290020
2,BA,Acajutiba,290030
3,BA,Adustina,290035
4,BA,Água Fria,290040
...,...,...,...
811,PR,Verê,412860
812,PR,Virmond,412865
813,PR,Vitorino,412870
814,PR,Wenceslau Braz,412850


In [8]:
# Carregar dataset em CSV

df_bapr = pd.read_csv('/kaggle/input/bitos-bahia-e-paran-entre-2021-e-2023/df2123_ba_pr.csv')

In [9]:
# Notas do professor:

'''
# Faça esse código no CHAT

complete=NULL

for(i in 1:24){
complete[i]=100*(table(dados[,i]=="NA")/ 783239)
}
# Temos 783239 valores?
> complete
 [1]  97.573793 100.000000 100.000000  98.421427 100.000000  10.531268  99.556457 100.000000  97.103566  95.624579  94.826483  93.962507  87.348434 100.000000
[15] 100.000000  70.042094   1.801749   1.922402 100.000000 100.000000 100.000000 100.000000 100.000000 100.000000

Percebemos que, tem variáveis com 1.80% completo dos dados não
devemos trabalhar com essas.

Podemos ficar com variáveis no limite de 85% para cima para aplicar o drop

                   complete
 [1,] "CODMUNNATU" "97.5737929291059"  Ok, pensar em cidades
 [2,] "CODMUNOCOR" "100"    ok   ,  pensar em cidades
 [3,] "CODMUNRES"  "100"   ok
 [4,] "NATURAL"    "98.4214269207739"ok, pensar em agrupar
 [5,] "CAUSABAS"   "100"   ok  , MAS muitas categorias
 [6,] "CIRCOBITO"  "10.5312682335788" Aqui está o suicicio ? temos que pensar
 [7,] "IDADE"      "99.5564572244232" ok
 [8,] "SEXO"       "100"         ok
 [9,] "RACACOR"    "97.1035660890226" ok
[10,] "ESTCIV"     "95.6245794706341" ok
[11,] "ESC"        "94.8264833594854" ok
[12,] "ESCFALAGR1" "93.9625069742441" ok
[13,] "OCUP"       "87.3484338752284" ok , temos que pensar como agripar
[14,] "DTOBITO"    "100"       tem sentido  ?
[15,] "DIFDATA"    "100"       tem sentido  ?
[16,] "NECROPSIA"  "70.0420944309464" se aplicarmos o drop, perdemos 30% dos dados
[17,] "QTDFILMORT" "1.80174889146225" retira
[18,] "GRAVIDEZ"   "1.92240171901552" retira
[19,] "LINHAA"     "100"      a partir daqui essas variáveis  muitas categorias, faz sentido?
[20,] "LINHAB"     "100"
[21,] "LINHAC"     "100"
[22,] "LINHAD"     "100"
[23,] "LINHAII"    "100"
[24,] "ATESTADO"   "100"
'''

'\n# Faça esse código no CHAT\n\ncomplete=NULL\n\nfor(i in 1:24){\ncomplete[i]=100*(table(dados[,i]=="NA")/ 783239)\n}\n# Temos 783239 valores?\n> complete\n [1]  97.573793 100.000000 100.000000  98.421427 100.000000  10.531268  99.556457 100.000000  97.103566  95.624579  94.826483  93.962507  87.348434 100.000000\n[15] 100.000000  70.042094   1.801749   1.922402 100.000000 100.000000 100.000000 100.000000 100.000000 100.000000\n\nPercebemos que, tem variáveis com 1.80% completo dos dados não\ndevemos trabalhar com essas.\n\nPodemos ficar com variáveis no limite de 85% para cima para aplicar o drop\n\n                   complete          \n [1,] "CODMUNNATU" "97.5737929291059"  Ok, pensar em cidades\n [2,] "CODMUNOCOR" "100"    ok   ,  pensar em cidades      \n [3,] "CODMUNRES"  "100"   ok          \n [4,] "NATURAL"    "98.4214269207739"ok, pensar em agrupar \n [5,] "CAUSABAS"   "100"   ok  , MAS muitas categorias\n [6,] "CIRCOBITO"  "10.5312682335788" Aqui está o suicicio ? temos que 

In [10]:
# Reorganizar df_bapr em um novo dataframe
df = df_bapr[
    ['CODMUNNATU',
    'CODMUNOCOR',
    'CODMUNRES',
    'NATURAL',
    'CAUSABAS',
    'IDADE',
    'SEXO',
    'RACACOR',
    'ESTCIV',
    'ESC',
    'ESCFALAGR1',
    'OCUP',
    'NECROPSIA'
    ]
]

In [11]:
df.head()

,CODMUNNATU,CODMUNOCOR,CODMUNRES,NATURAL,CAUSABAS,IDADE,SEXO,RACACOR,ESTCIV,ESC,ESCFALAGR1,OCUP,NECROPSIA
0,120020.0,410690,120020,812.0,C258,460.0,2,1.0,2.0,3.0,11.0,999992.0,2.0
1,411610.0,120040,120040,841.0,B342,455.0,2,1.0,2.0,5.0,8.0,241005.0,1.0
2,412650.0,120040,120040,841.0,B342,485.0,1,1.0,2.0,3.0,11.0,354705.0,1.0
3,411190.0,110020,120040,841.0,B342,471.0,1,1.0,NaN,NaN,NaN,782510.0,2.0
4,412650.0,120040,120040,841.0,B342,479.0,2,1.0,2.0,4.0,12.0,NaN,1.0


In [14]:
# Retire essas aspas para baixar os novos dataframes no pc pessoal


# Baixar o dataframe em CSV para PC

# Importar biblioteca
from google.colab import files

# Definir qual dataframe você quer baixar (neste caso é o df2123)
df_baixar_i = 'df'
df_baixar_ii = 'mun_bapr'

# Save DataFrame to CSV file
df.to_csv('df.csv', index=False)

# Download CSV
files.download('df.csv')

# Save DataFrame to CSV file
mun_bapr.to_csv('mun_bapr.csv', index=False)

# Download CSV
files.download('mun_bapr.csv')

# Retire essas aspas

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>